# Packages that need to be installed initially


In [ ]:
sc.install_pypi_package("pandas==0.25.1")
# sc.list_packages()

In [6]:
import pyspark
import pandas as pd
from pyspark.sql.functions import split, round
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Property Search City

In [9]:
city = "lehigh acres"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read property data from S3 Bucket (infer schema and set headers = true)
### Print dataframe schema

In [10]:
spark_properties_df = spark.read.csv(f"s3://real-estate-extracted-data/property_search/properties-for-sale-{city}-fl.csv",
                               inferSchema=True, header=True)
spark_properties_df.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dateSold: string (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- lotAreaValue: double (nullable = true)
 |-- address: string (nullable = true)
 |-- daysOnZillow: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- listingDateTime: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- contingentListingType: string (nullable = true)
 |-- listingStatus: string (nullable = true)
 |-- zpid: integer (nullable = true)
 |-- imgSrc: string (nullable = true)
 |-- livingArea: double (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- lotAreaUnit: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- hasImage: boolean (nullable = true)
 |-- listingSubType.is_FSBA: boolean (nullable = true)
 |-- listingSubType.is_newHome: boolean (nullable = true)
 |-- listingSubType.is_openHouse: boolean (

# Replacing '.' with '_' in column headers.
### Doing this allows me to select 'listingSubType_' columns easier and cleaner

In [183]:
clean_properties_df = spark_properties_df.toDF(*(c.replace('.', '_') for c in spark_properties_df.columns))

clean_properties_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dateSold: string (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- lotAreaValue: double (nullable = true)
 |-- address: string (nullable = true)
 |-- daysOnZillow: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- listingDateTime: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- contingentListingType: string (nullable = true)
 |-- listingStatus: string (nullable = true)
 |-- zpid: integer (nullable = true)
 |-- imgSrc: string (nullable = true)
 |-- livingArea: double (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- lotAreaUnit: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- hasImage: boolean (nullable = true)
 |-- listingSubType_is_FSBA: boolean (nullable = true)
 |-- listingSubType_is_newHome: boolean (nullable = true)
 |-- listingSubType_is_openHouse: boolean (

# Convert spark df to pandas 
### (Used after changes made to 'clean_properties_df'. Only way to display data nicely.)

In [178]:
pandas_df = clean_properties_df.toPandas()
pd.set_option('display.max_columns', None)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
pandas_df.head(40)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     propertyType  lotAreaValue  \
0   SINGLE_FAMILY         0.250   
1   SINGLE_FAMILY         0.252   
2   SINGLE_FAMILY         0.292   
3   SINGLE_FAMILY         0.359   
4   SINGLE_FAMILY      9426.384   
5   SINGLE_FAMILY         0.250   
6   SINGLE_FAMILY         0.507   
7   SINGLE_FAMILY         0.284   
8   SINGLE_FAMILY         0.287   
9   SINGLE_FAMILY         0.000   
10  SINGLE_FAMILY         0.250   
11   MULTI_FAMILY      4791.600   
12  SINGLE_FAMILY      9888.120   
13  SINGLE_FAMILY      8929.800   
14  SINGLE_FAMILY     10236.600   
15  SINGLE_FAMILY         0.500   
16  SINGLE_FAMILY     10672.200   
17  SINGLE_FAMILY     10802.880   
18  SINGLE_FAMILY         0.250   
19  SINGLE_FAMILY      8363.520   
20  SINGLE_FAMILY     10628.640   
21   MULTI_FAMILY      1698.840   
22  SINGLE_FAMILY      8581.320   
23  SINGLE_FAMILY      8480.000   
24  SINGLE_FAMILY         0.250   
25  SINGLE_FAMILY      6795.360   
26   MULTI_FAMILY      6229.080   
27  SINGLE_FAMILY   

# Drop 'dateSold' column

In [184]:
clean_properties_df = clean_properties_df.drop('dateSold')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Filtering properties by     
    # bathsMin = 2
    # bedsMax = 4

In [185]:
clean_properties_df = clean_properties_df.filter((clean_properties_df["bedrooms"] < 5) &
                     (clean_properties_df["bathrooms"] >= 2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Dropping Rows...
### where homes have NaN lot area or NaN living area (PLAN homes) 

In [186]:
clean_properties_df = clean_properties_df.na.drop(how='any', subset=['lotAreaValue', 'livingArea'])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### where  'newHome' is equal to True (I'm looking for fixer-upper's)

In [187]:
# Fill null values with 'False' value
clean_properties_df = clean_properties_df.fillna({'listingSubType_is_newHome': 'False'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [188]:
clean_properties_df = clean_properties_df.where("listingSubType_is_newHome == False")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Creating Column for Street and City based off of address (just to demonstrate ability)

In [189]:
clean_properties_df = clean_properties_df.withColumn("Street", split(F.col("address"), ",").getItem(0)).withColumn("City", split(F.col("address"), ",").getItem(1))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Orderby price lowest to highest then orderby living area highest to lowest

In [190]:
clean_properties_df = clean_properties_df.orderBy(F.col("price").asc(), F.col("livingArea").desc())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Groupby 
### Bed and bath (count)

In [13]:
clean_properties_df.groupby(['bedrooms', 'bathrooms']).count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-----+
|bedrooms|bathrooms|count|
+--------+---------+-----+
|       3|        2|   19|
|       3|        3|    1|
|       4|        2|    4|
|       2|        2|    8|
+--------+---------+-----+

### Bed and bath (avg price)

In [201]:
clean_properties_df.groupby(['bedrooms', 'bathrooms']).avg('price').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-----------------+
|bedrooms|bathrooms|       avg(price)|
+--------+---------+-----------------+
|       3|        2|261684.1052631579|
|       3|        3|         295900.0|
|       4|        2|         277722.5|
|       2|        2|       217562.375|
+--------+---------+-----------------+

### Bed and bath (avg livingArea)

In [233]:
clean_properties_df.groupby(['bedrooms', 'bathrooms']).avg('livingArea').sort('avg(livingArea)').show()

  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+------------------+
|bedrooms|bathrooms|   avg(livingArea)|
+--------+---------+------------------+
|       2|        2|          1069.125|
|       3|        2|1318.9473684210527|
|       4|        2|            1510.5|
|       3|        3|            1664.0|
+--------+---------+------------------+

### PropertyType, Bed, and bath (count)

In [237]:
clean_properties_df.groupby(['propertyType', 'bedrooms', 'bathrooms']).count().sort('count').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+---------+-----+
| propertyType|bedrooms|bathrooms|count|
+-------------+--------+---------+-----+
| MULTI_FAMILY|       3|        2|    1|
|SINGLE_FAMILY|       3|        3|    1|
| MULTI_FAMILY|       2|        2|    2|
|SINGLE_FAMILY|       4|        2|    4|
|SINGLE_FAMILY|       2|        2|    6|
|SINGLE_FAMILY|       3|        2|   18|
+-------------+--------+---------+-----+

### PropertyType, Bed, and bath (avg living area)

In [235]:
clean_properties_df.groupby(['propertyType', 'bedrooms', 'bathrooms']).avg('livingArea').sort('avg(livingArea)').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+---------+------------------+
| propertyType|bedrooms|bathrooms|   avg(livingArea)|
+-------------+--------+---------+------------------+
|SINGLE_FAMILY|       2|        2|1036.3333333333333|
| MULTI_FAMILY|       2|        2|            1167.5|
|SINGLE_FAMILY|       3|        2|1304.8333333333333|
|SINGLE_FAMILY|       4|        2|            1510.5|
| MULTI_FAMILY|       3|        2|            1573.0|
|SINGLE_FAMILY|       3|        3|            1664.0|
+-------------+--------+---------+------------------+

### Homes for auction (count)

In [172]:
clean_properties_df.groupby('listingSubType_is_forAuction').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+-----+
|listingSubType_is_forAuction|count|
+----------------------------+-----+
|                        null|   33|
|                        true|    1|
+----------------------------+-----+

# Write Spark DF to S3 bucket as a CSV

In [1]:
# coalesce(1) writes csv to one file in S3
clean_properties_df.coalesce(1) \
    .write.format('csv') \
    .mode('overwrite') \
    .option("header", "true") \
    .save(f's3://real-estate-filtered-data/property-prospects/{city}.csv/')


VBox()

Starting Spark application


KeyboardInterrupt: 